In [1]:
import numpy as np
import tensorflow as tf
import torch as th

In [2]:
np.__version__

'1.16.5'

In [3]:
tf.__version__

'2.1.0'

In [4]:
th.__version__

'1.4.0'

In [34]:
import pandas as pd

In [5]:
tf.config.set_visible_devices([], 'GPU')

In [6]:
X = np.random.randn(100,5)

In [7]:
X.shape

(100, 5)

In [9]:
X[:10,:]

array([[ 1.4724413 , -0.20693254,  1.8911695 ,  0.01344768, -0.85709534],
       [-0.15302793, -2.26994167, -0.0777953 , -1.02104214, -0.67252393],
       [-0.72153483, -0.76947224,  1.42082275, -0.0618513 , -1.48820933],
       [-0.3644801 , -1.38207133, -0.10794663, -1.59402099,  1.24163734],
       [-2.09322333,  0.16440003, -0.79866581, -0.01684925, -0.394218  ],
       [-0.12433771, -0.24219916,  1.27051678,  0.46489733, -0.39925367],
       [-1.36190415,  1.65805189, -0.17231741, -0.21647334,  0.93649158],
       [-0.08338457, -0.222076  , -0.18900341,  1.62804291,  0.10137476],
       [ 0.71557195, -0.19558445, -0.61039342,  0.51209783, -0.19807739],
       [ 0.6599818 , -0.48191418,  0.77877636, -0.67017601,  1.10167351]])

In [17]:
X_bias = np.concatenate((X, np.ones(shape=(100,1))), axis=1)

In [18]:
X_bias[:10]

array([[ 1.4724413 , -0.20693254,  1.8911695 ,  0.01344768, -0.85709534,
         1.        ],
       [-0.15302793, -2.26994167, -0.0777953 , -1.02104214, -0.67252393,
         1.        ],
       [-0.72153483, -0.76947224,  1.42082275, -0.0618513 , -1.48820933,
         1.        ],
       [-0.3644801 , -1.38207133, -0.10794663, -1.59402099,  1.24163734,
         1.        ],
       [-2.09322333,  0.16440003, -0.79866581, -0.01684925, -0.394218  ,
         1.        ],
       [-0.12433771, -0.24219916,  1.27051678,  0.46489733, -0.39925367,
         1.        ],
       [-1.36190415,  1.65805189, -0.17231741, -0.21647334,  0.93649158,
         1.        ],
       [-0.08338457, -0.222076  , -0.18900341,  1.62804291,  0.10137476,
         1.        ],
       [ 0.71557195, -0.19558445, -0.61039342,  0.51209783, -0.19807739,
         1.        ],
       [ 0.6599818 , -0.48191418,  0.77877636, -0.67017601,  1.10167351,
         1.        ]])

In [21]:
y = np.random.randn(100,1)

In [22]:
X_bias.shape

(100, 6)

In [23]:
y.shape

(100, 1)

In [48]:
lmbd = 0.0 # controls overfit

In [49]:
# Theta = (X^T.X)^(-1).(X^T).y
np_theta = np.linalg.inv(X_bias.T.dot(X_bias) + lmbd * np.eye(X_bias.shape[1])).dot(X_bias.T).dot(y)

In [50]:
np_theta

array([[-0.10586276],
       [-0.01440132],
       [ 0.03569265],
       [ 0.09521986],
       [ 0.0561542 ],
       [-0.04208463]])

In [32]:
def np_predict(x_test, theta):
    return x_test.dot(theta)

In [33]:
def test_mae(x_test, y_test, theta):
    y_pred = np_predict(x_test, theta)
    mae = np.abs(y_test - y_pred).mean()
    return mae

In [35]:
def test_analysis(x_test, y_test, theta):
    mae = test_mae(x_test, y_test, theta)
    y_pred = np_predict(x_test, theta)
    print("Mean abs error: {:.1f}".format(mae))
    print("A few preds vs test reality:")
    df_result = pd.DataFrame({'reality' : y_test, 'predictions':y_pred})
    print(df_result.head(20))

In [39]:
tf_X_bias = tf.constant(X_bias)
tf_X_bias[:5]

<tf.Tensor: shape=(5, 6), dtype=float64, numpy=
array([[ 1.4724413 , -0.20693254,  1.8911695 ,  0.01344768, -0.85709534,
         1.        ],
       [-0.15302793, -2.26994167, -0.0777953 , -1.02104214, -0.67252393,
         1.        ],
       [-0.72153483, -0.76947224,  1.42082275, -0.0618513 , -1.48820933,
         1.        ],
       [-0.3644801 , -1.38207133, -0.10794663, -1.59402099,  1.24163734,
         1.        ],
       [-2.09322333,  0.16440003, -0.79866581, -0.01684925, -0.394218  ,
         1.        ]])>

In [40]:
tf_y = tf.constant(y)
tf_y[:5]

<tf.Tensor: shape=(5, 1), dtype=float64, numpy=
array([[ 1.16381326],
       [ 0.06101082],
       [-1.27123443],
       [-1.54717364],
       [ 0.11969632]])>

In [46]:
# Theta = (X^T.X)^(-1).(X^T).y

tf_theta = tf.matmul(tf.matmul(a=tf.linalg.inv(tf.matmul(tf_X_bias, tf_X_bias, transpose_a=True)),b=tf_X_bias, transpose_b=True) , tf_y)

In [47]:
tf_theta

<tf.Tensor: shape=(6, 1), dtype=float64, numpy=
array([[-0.10586276],
       [-0.01440132],
       [ 0.03569265],
       [ 0.09521986],
       [ 0.0561542 ],
       [-0.04208463]])>

In [51]:
np_theta

array([[-0.10586276],
       [-0.01440132],
       [ 0.03569265],
       [ 0.09521986],
       [ 0.0561542 ],
       [-0.04208463]])

In [52]:
th_dev = th.device('cuda')

In [53]:
th_dev

device(type='cuda')

In [56]:
th.cuda.get_device_name(th_dev)

'GeForce GTX 1070 with Max-Q Design'

In [58]:
th.cuda.memory_allocated(th_dev)

0

In [61]:
th.cuda.get_device_properties(th_dev)

_CudaDeviceProperties(name='GeForce GTX 1070 with Max-Q Design', major=6, minor=1, total_memory=8192MB, multi_processor_count=16)

In [65]:
th_X_bias = th.tensor(X_bias)
th_y = th.tensor(y)
th_X_bias[:5]

tensor([[ 1.4724, -0.2069,  1.8912,  0.0134, -0.8571,  1.0000],
        [-0.1530, -2.2699, -0.0778, -1.0210, -0.6725,  1.0000],
        [-0.7215, -0.7695,  1.4208, -0.0619, -1.4882,  1.0000],
        [-0.3645, -1.3821, -0.1079, -1.5940,  1.2416,  1.0000],
        [-2.0932,  0.1644, -0.7987, -0.0168, -0.3942,  1.0000]],
       dtype=torch.float64)

In [66]:
th_X_bias = th_X_bias.to(th_dev)
th_y = th_y.to(th_dev)
th_X_bias[:5]

tensor([[ 1.4724, -0.2069,  1.8912,  0.0134, -0.8571,  1.0000],
        [-0.1530, -2.2699, -0.0778, -1.0210, -0.6725,  1.0000],
        [-0.7215, -0.7695,  1.4208, -0.0619, -1.4882,  1.0000],
        [-0.3645, -1.3821, -0.1079, -1.5940,  1.2416,  1.0000],
        [-2.0932,  0.1644, -0.7987, -0.0168, -0.3942,  1.0000]],
       device='cuda:0', dtype=torch.float64)

In [70]:
th_theta = th.matmul(
    th.matmul(
        th.inverse(
            th.matmul(
                th.transpose(th_X_bias, 0,1), 
                th_X_bias)
        ),
        th.transpose(th_X_bias, 0, 1)
    ), 
    th_y)

In [71]:
th_theta

tensor([[-0.1059],
        [-0.0144],
        [ 0.0357],
        [ 0.0952],
        [ 0.0562],
        [-0.0421]], device='cuda:0', dtype=torch.float64)

In [75]:
th_cpu_theta = th_theta.cpu()

In [79]:
th_cpu_theta.numpy()

array([[-0.10586276],
       [-0.01440132],
       [ 0.03569265],
       [ 0.09521986],
       [ 0.0561542 ],
       [-0.04208463]])

In [80]:
np_theta

array([[-0.10586276],
       [-0.01440132],
       [ 0.03569265],
       [ 0.09521986],
       [ 0.0561542 ],
       [-0.04208463]])

In [81]:
np_theta[0,0]

-0.10586275799573999

In [82]:
th_cpu_theta.numpy()[0,0]

-0.10586275799574006

In [ ]:
# TODO
# load imobiliare.ro scrapped data
# best way: aggregate multiple sources of scrapped data (multiple student sources)
# keep first 80% for train and last 20% for test - NOT the correct way to do it
# m = train.shape[0]
# train_size = int(m * 0.8)
# test_size = int(m *0.2)
# train_data = all_data[:-train_size]
# test_data = all_data[-train_size:]
# "train" np, tf, th models
# test them!